In [ ]:
import pandas as pd
import re
from datetime import datetime, timedelta

df = pd.read_excel(r'G:\desktop\data分词.xlsx')

column_name = 'Date'  

def extract_ip(text):
    ip_pattern = r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})'
    match = re.search(ip_pattern, text)
    return match.group(0) if match else None

def extract_and_format_date(text):
    if '编辑于' in text:
        date_pattern = r'编辑于 (\d{1,2}-\d{1,2})'
        date_match = re.search(date_pattern, text)
        if date_match:
            date_str = date_match.group(1)
            year = datetime.now().year
            return datetime.strptime(f'{year}-{date_str}', '%Y-%m-%d')
        
        day_pattern = r'编辑于 (\d+) 天前'
        day_match = re.search(day_pattern, text)
        if day_match:
            days_ago = int(day_match.group(1))
            return datetime.now() - timedelta(days=days_ago)
    
    standard_date_pattern = r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})'
    standard_date_match = re.search(standard_date_pattern, text)
    if standard_date_match:
        return datetime.strptime(standard_date_match.group(1), '%Y-%m-%d %H:%M:%S')
    
    return None

df['IP'] = df[column_name].apply(extract_ip)
df['Date'] = df[column_name].apply(extract_and_format_date)

df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

df.to_excel('processed_file.xlsx', index=False)

print(df.head())
